In [1]:
import yaml
import click
import torch
import tqdm.auto
import numpy as np
 
import sys
import os
import math
#project_root = "C:/Users/elmop/deep_feature_reweighting/deep_feature_reweighting/external/pruning_by_explaining"
project_root = "/home/primmere/ide/external/pruning_by_explaining"
sys.path.insert(0, project_root)                 
sys.path.insert(0, os.path.dirname(project_root))

from pruning_by_explaining.models import ModelLoader
from pruning_by_explaining.metrics import compute_accuracy
from pruning_by_explaining.my_metrics import compute_worst_accuracy
from pruning_by_explaining.my_datasets import WaterBirds, get_sample_indices_for_group, WaterBirdSubset, ISIC, ISICSubset


from pruning_by_explaining.pxp import (
    ModelLayerUtils,
    get_cnn_composite,
    get_vit_composite,
)

from pruning_by_explaining.pxp import GlobalPruningOperations
from pruning_by_explaining.pxp import ComponentAttibution

In [2]:
num_workers = 4
device = torch.device("cuda")
device_string = "cuda"
#waterbirds = WaterBirds("F:/model_stuff/waterbirds/waterbird", seed = 1, num_workers = num_workers)
waterbirds = WaterBirds('/scratch_shared/primmere/waterbird', seed = 1, num_workers = num_workers)

val_set = waterbirds.get_valid_set()

pruning_indices = get_sample_indices_for_group(val_set, 10, device_string, [3])
validation_indices = get_sample_indices_for_group(val_set, 'all', device_string)

custom_pruning_set = WaterBirdSubset(val_set, pruning_indices)
custom_val_set = WaterBirdSubset(val_set, validation_indices)

prune_dataloader = torch.utils.data.DataLoader(custom_pruning_set, batch_size=1, shuffle=True, num_workers=num_workers)
val_dataloader = torch.utils.data.DataLoader(custom_val_set, batch_size=8, shuffle=True, num_workers=num_workers)

[0 1 2 3]
Number of unique labels: 2, Number of unique places: 2, Total groups: 4
group 0: 456
group 1: 456
group 2: 143
group 3: 144
target groups: [3]
target groups: [0, 1, 2, 3]


In [3]:
suggested_composite = {
        "low_level_hidden_layer_rule": "Epsilon",
        "mid_level_hidden_layer_rule":"Epsilon",
        "high_level_hidden_layer_rule": "Epsilon",
        "fully_connected_layers_rule": "Epsilon",
        "softmax_rule": "Epsilon",
    }
composite = get_vit_composite("vit_b_16", suggested_composite)

#model = ModelLoader.get_basic_model("vit_b_16", "F:/model_stuff/vit_waterbirds.pth", torch.device("cpu"), num_classes=2)
model = ModelLoader.get_basic_model("vit_b_16", "/home/primmere/ide/dfr/logs/vit_waterbirds.pth", device, num_classes=2)



Arch:vit_b_16


/home/primmere/ide/external/pruning_by_explaining/models/utils.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_checkpoint = torch.load(checkpoint_path, map_locat

In [4]:
layer_types = {
        "Softmax": torch.nn.Softmax,
        "Linear": torch.nn.Linear,
        "Conv2d": torch.nn.Conv2d,
    }

In [5]:
print("hi")
acc_top1 = compute_accuracy(
        model,
        val_dataloader,
        device,
    )
print(acc_top1)

hi


evaluating acc:   0%|          | 0/150 [00:00<?, ?it/s]

0.8807339449541285


In [6]:
acc_worst, acc_groups = compute_worst_accuracy(
        model,
        val_dataloader,
        device,
    )
for i in range(4):
    print(f'{i}: {acc_groups[i]}')


evaluating group acc:   0%|          | 0/150 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x14dcc306b370>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x14dcc306b370>Traceback (most recent call last):

  File "/home/primmere/.conda/envs/dfr2/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
Traceback (most recent call last):
  File "/home/primmere/.conda/envs/dfr2/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/home/primmere/.conda/envs/dfr2/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
  File "/home/primmere/.conda/envs/dfr2/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
        if w.is_alive():if w.is_alive():
  File "/home/primmere/.conda/envs/dfr2/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid =

0: 0.9868421052631579
1: 0.7916666666666666
2: 0.7272727272727273
3: 0.9791666666666666


In [7]:
acc_groups[0]

0.9868421052631579

In [8]:
len(validation_indices)

1199

In [9]:
component_attributor = ComponentAttibution(
        "Relevance",
        "ViT",
        layer_types['Linear'],
        True
    )
print("done")

zennit
done


In [10]:
components_relevances = component_attributor.attribute(
        model,
        prune_dataloader,
        composite,
        abs_flag=True,
        device=device,
    )


/home/primmere/.conda/envs/dfr2/lib/python3.10/site-packages/torch/autograd/graph.py:768: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [11]:
for k in components_relevances.keys():
    print(k, torch.mean(components_relevances[k]))

encoder.layers.encoder_layer_0.mlp.0 tensor(0.0397)
encoder.layers.encoder_layer_0.mlp.3 tensor(0.1729)
encoder.layers.encoder_layer_1.mlp.0 tensor(0.0504)
encoder.layers.encoder_layer_1.mlp.3 tensor(0.1234)
encoder.layers.encoder_layer_2.mlp.0 tensor(0.0670)
encoder.layers.encoder_layer_2.mlp.3 tensor(0.1473)
encoder.layers.encoder_layer_3.mlp.0 tensor(0.0592)
encoder.layers.encoder_layer_3.mlp.3 tensor(0.1161)
encoder.layers.encoder_layer_4.mlp.0 tensor(0.0551)
encoder.layers.encoder_layer_4.mlp.3 tensor(0.1032)
encoder.layers.encoder_layer_5.mlp.0 tensor(0.0559)
encoder.layers.encoder_layer_5.mlp.3 tensor(0.0981)
encoder.layers.encoder_layer_6.mlp.0 tensor(0.0583)
encoder.layers.encoder_layer_6.mlp.3 tensor(0.1000)
encoder.layers.encoder_layer_7.mlp.0 tensor(0.0494)
encoder.layers.encoder_layer_7.mlp.3 tensor(0.0992)
encoder.layers.encoder_layer_8.mlp.0 tensor(0.0369)
encoder.layers.encoder_layer_8.mlp.3 tensor(0.0802)
encoder.layers.encoder_layer_9.mlp.0 tensor(0.0374)
encoder.laye

In [12]:
layer_names = component_attributor.layer_names
pruner = GlobalPruningOperations(
        layer_types["Linear"],
        layer_names,
    )

In [13]:
global_pruning_mask = pruner.generate_global_pruning_mask(
                model,
                components_relevances,
                0.5,
                subsequent_layer_pruning="Linear",
                least_relevant_first=True,
                device=device,
            )

In [14]:
global_pruning_mask.keys()

odict_keys(['encoder.layers.encoder_layer_0.mlp.0', 'encoder.layers.encoder_layer_0.mlp.3', 'encoder.layers.encoder_layer_1.mlp.0', 'encoder.layers.encoder_layer_1.mlp.3', 'encoder.layers.encoder_layer_2.mlp.0', 'encoder.layers.encoder_layer_2.mlp.3', 'encoder.layers.encoder_layer_3.mlp.0', 'encoder.layers.encoder_layer_3.mlp.3', 'encoder.layers.encoder_layer_4.mlp.0', 'encoder.layers.encoder_layer_4.mlp.3', 'encoder.layers.encoder_layer_5.mlp.0', 'encoder.layers.encoder_layer_5.mlp.3', 'encoder.layers.encoder_layer_6.mlp.0', 'encoder.layers.encoder_layer_6.mlp.3', 'encoder.layers.encoder_layer_7.mlp.0', 'encoder.layers.encoder_layer_7.mlp.3', 'encoder.layers.encoder_layer_8.mlp.0', 'encoder.layers.encoder_layer_8.mlp.3', 'encoder.layers.encoder_layer_9.mlp.0', 'encoder.layers.encoder_layer_9.mlp.3', 'encoder.layers.encoder_layer_10.mlp.0', 'encoder.layers.encoder_layer_10.mlp.3', 'encoder.layers.encoder_layer_11.mlp.0', 'encoder.layers.encoder_layer_11.mlp.3'])

In [15]:
hook_handles = pruner.fit_pruning_mask(
                model, global_pruning_mask,
            )
print(hook_handles)

None


In [16]:
acc_worst, acc_groups = compute_worst_accuracy(
        model,
        val_dataloader,
        device,
    )
for i in range(4):
    print(f'{i}: {acc_groups[i]}')


evaluating group acc:   0%|          | 0/150 [00:00<?, ?it/s]

0: 0.956140350877193
1: 0.19298245614035087
2: 0.6503496503496503
3: 0.9791666666666666


In [17]:
global_pruning_mask2 = pruner.generate_global_pruning_mask(
                model,
                components_relevances,
                0.8,
                subsequent_layer_pruning="Linear",
                least_relevant_first=True,
                device=device,
            )

In [18]:
global_pruning_mask2

OrderedDict([('encoder.layers.encoder_layer_0.mlp.0',
              {'Linear': {'weight': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [1., 1., 1.,  ..., 1., 1., 1.],
                        ...,
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [1., 1., 1.,  ..., 1., 1., 1.],
                        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'),
                'bias': tensor([0., 0., 1.,  ..., 0., 1., 0.], device='cuda:0')}}),
             ('encoder.layers.encoder_layer_0.mlp.3',
              {'Linear': {'weight': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
                        [1., 1., 1.,  ..., 1., 1., 1.],
                        [1., 1., 1.,  ..., 1., 1., 1.],
                        ...,
                        [1., 1., 1.,  ..., 1., 1., 1.],
                        [1., 1., 1.,  ..., 1., 1., 1.],
                        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0'),


In [19]:
model.encoder.layers[0].mlp[0].weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
        [-0.0071, -0.0030, -0.0284,  ...,  0.0062, -0.0066,  0.0229],
        ...,
        [-0.0000, -0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
        [ 0.0046, -0.0134,  0.0103,  ...,  0.0616,  0.0905, -0.0828],
        [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', requires_grad=True)

In [20]:
hook_handles = pruner.fit_pruning_mask(
                model, global_pruning_mask2,
            )
print(hook_handles)

None


In [21]:
acc_worst, acc_groups = compute_worst_accuracy(
        model,
        val_dataloader,
        device,
    )
for i in range(4):
    print(f'{i}: {acc_groups[i]}')


evaluating group acc:   0%|          | 0/150 [00:00<?, ?it/s]

0: 0.47149122807017546
1: 0.05043859649122807
2: 0.7062937062937062
3: 0.9652777777777778
